In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
!pip install ipywidgets --quiet
from IPython.display import display
import ipywidgets as widgets
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
class EMGLSTMClassifier(nn.Module):
    def __init__(self, inputSize=8, latentDim=64, layers=1, classes=8, dropout=0.3):
        super(EMGLSTMClassifier, self).__init__()

        self.lstm = nn.LSTM(input_size=inputSize,
                            hidden_size=latentDim,
                            num_layers=layers,
                            batch_first=True,
                            dropout=dropout)

        self.fc1 = nn.Linear(latentDim, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [3]:
modelPath = "/content/drive/MyDrive/ConcentracionIA/Periodo2/Mod2. Tecnicas y arquitecturas de deep learning/Proyecto/PortafolioImplementacionMod2/Modelo/EMGLSTMModel.pt"
scalerPath = "/content/drive/MyDrive/ConcentracionIA/Periodo2/Mod2. Tecnicas y arquitecturas de deep learning/Proyecto/PortafolioImplementacionMod2/Scaler/scaler.save"

scaler = joblib.load(scalerPath)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EMGLSTMClassifier()
model.load_state_dict(torch.load(modelPath, map_location=device))
model.to(device)
model.eval()

print("Modelo y scaler cargados")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Modelo y scaler cargados


In [4]:
uploadedFile = {}
output = widgets.Output()

def fileUpload(change):
    global uploadedFile
    uploadedFile = change['new']
    with output:
        output.clear_output()
        if uploadedFile:
            name = list(uploadedFile.keys())[0]
            print(f"Archivo cargado: {name}")
        else:
            print("No se seleccionó ningún archivo")

widget = widgets.FileUpload(
    accept='.txt',
    multiple=False,
    description='Cargar .txt'
)

widget.observe(fileUpload, names='value')

In [5]:
gestureLabels = {
    1: "Mano relajada",
    2: "Puño cerrado",
    3: "Flexión",
    4: "Extensión",
    5: "Desviación radial",
    6: "Desviación cubital",
    7: "Palma extendida"
}

In [6]:
predictButton = widgets.Button(
    description='Predecir',
    button_style='success'
)

output = widgets.Output()

In [7]:
def predictClicked(b):
    with output:
        output.clear_output()

        if not uploadedFile:
            print("Carga primero un archivo .txt")
            return

        try:
            name = list(uploadedFile.keys())[0]
            dataFile = uploadedFile[name]
            content = dataFile['content']

            EMGtxt = pd.read_csv(io.BytesIO(content),
                             sep=r"\s+", engine="python",
                             header=None, on_bad_lines='skip')

            if EMGtxt.shape[1] != 10:
                print(f"El archivo tiene {EMGtxt.shape[1]} columnas (deben ser 10).")
                return

            EMGtxt.columns = ['time'] + [f'ch{i}' for i in range(1, 9)] + ['label']
            EMGtxt = EMGtxt[EMGtxt['label'] != 0].reset_index(drop=True)
            EMGtxt = EMGtxt.apply(pd.to_numeric, errors='coerce').dropna().reset_index(drop=True)

            classCounts = EMGtxt['label'].value_counts()
            classCounts = classCounts[classCounts.index != 0]

            classPick = classCounts.idxmax()
            gesture = EMGtxt[EMGtxt['label'] == classPick]

            if len(gesture) < 200:
                print(f"No hay suficientes muestras del gesto {classPick} (solo {len(gesture)}).")
                return

            window = gesture.iloc[:200][[f'ch{i}' for i in range(1, 9)]].values
            windowScaled = scaler.transform(window)
            inputTensor = torch.tensor(windowScaled, dtype=torch.float32).unsqueeze(0).to(device)

            with torch.no_grad():
                outputModel = model(inputTensor)
                predClass = torch.argmax(outputModel, dim=1).item()

            predLabel = gestureLabels.get(predClass, f"{predClass} (desconocido)")
            realLabel = gestureLabels.get(classPick, f"{classPick} (desconocido)")

            print("======== Prediccion :D ========")
            print(f"Clase real: {classPick} — {realLabel}")
            print(f"Clase predicha por el modelo: {predClass} — {predLabel}")

        except Exception as e:
            print(f"Error al predecir: {e}")


In [8]:
predictButton.on_click(predictClicked)

display(widgets.VBox([
    widget,
    predictButton,
    output
]))